In [1]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

In [3]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [4]:
openai = OpenAI()

In [5]:
message = "Is this thing working?"
response = openai.chat.completions.create(model="gpt-4o-mini", messages=[{"role":"user", "content":message}])
print(response.choices[0].message.content)

Yes, it's working! How can I assist you today?


In [6]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [7]:
ed = Website("https://alessandrofugazza.com/")
print(ed.title)
print(ed.text)

Welcome to the Playground - AlessandroFugazza
AlessandroFugazza
Search
Home
Blog
About Me
Contact
GitHub
LinkedIn
Welcome to the Playground
A personal website I set up to learn WordPress.
Might as well use it to though.
What I’m doing
Robotics
Web Development
Machine Learning
Communication Technologies
iOS Development
3D Printing
AlessandroFugazza
${myTagline}. Just kidding, I can code.


In [8]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [9]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [10]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [15]:
def summarize(url):
    print("SUMMARIZING")
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [12]:
summarize("https://alessandrofugazza.com/")

'# Summary of "Welcome to the Playground - AlessandroFugazza"\n\nThis personal website, created by Alessandro Fugazza, serves as a platform for learning WordPress and showcases his interests and skills in various fields. Key areas of focus include:\n\n- Robotics\n- Web Development\n- Machine Learning\n- Communication Technologies\n- iOS Development\n- 3D Printing\n\nThe site appears to emphasize Alessandro\'s coding abilities with a light-hearted tagline that suggests a playful approach to learning and sharing his work.'

In [13]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [14]:
display_summary("https://alessandrofugazza.com/")

# Summary of "Welcome to the Playground - AlessandroFugazza"

The website "Welcome to the Playground" is a personal platform created by Alessandro Fugazza, primarily to learn WordPress. It serves multiple purposes, showcasing his interests and activities in various fields including:

- Robotics
- Web Development
- Machine Learning
- Communication Technologies
- iOS Development
- 3D Printing

The playful tagline included on the site suggests a sense of humor about coding skills. Additionally, there are sections for a blog and contact information, implying a potential for sharing insights and updates related to his projects. There are no specific news or announcements mentioned on the site.